In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from osgeo import gdal
from osgeo import gdalconst
import os
from osgeo import ogr
from osgeo import osr
import fiona
from ops.ops import load_json, load_opt_image
from tqdm.notebook import tqdm_notebook
from osgeo import gdal_array
from skimage.morphology import disk, dilation, erosion
import numpy as np
from s2cloudless import get_s2_evalscript, S2PixelCloudDetector

In [2]:
#image_folder = 'img/GEE-imgs'

paths = load_json(os.path.join('conf', 'paths.json'))
conf = load_json(os.path.join('conf', 'conf.json'))



In [8]:
image_folder = os.path.join('img', 'opt')
labels_folder = os.path.join('img', 'labels')
split_tiles = [6, 3]
val_tiles = [0, 2, 7, 9, 12,13] 

opt_file = os.path.join(image_folder, '2017', f'PA_S2_2017_B1_B2_B3_crop.tif')
opt_data = load_opt_image(opt_file)

shape = opt_data.shape[0:2]
print(shape)

tiles = np.zeros(shape, dtype=np.uint8).reshape((-1,1))
idx_matrix = np.arange(shape[0]*shape[1], dtype=np.uint32).reshape(shape)

tiles_idx = []
for hor in np.array_split(idx_matrix, split_tiles[0], axis=0):
    for tile in np.array_split(hor, split_tiles[1], axis=1):
        tiles_idx.append(tile)

   
for i, tile in enumerate(tiles_idx):
    if i in val_tiles:
        tiles[tile] = 1


tiles = tiles.reshape(shape)

data = gdal.Open(opt_file, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize

crs = data.GetSpatialRef()
proj = data.GetProjection()

pixel_width = geo_transform[1]

output_cmap = os.path.join(labels_folder, 'tiles.tif')

target_ds_cmap = gdal.GetDriverByName('GTiff').Create(output_cmap, x_res, y_res, 1, gdal.GDT_Byte)

target_ds_cmap.SetGeoTransform(geo_transform)
target_ds_cmap.SetSpatialRef(crs)
target_ds_cmap.SetProjection(proj)
band_cmap = target_ds_cmap.GetRasterBand(1)
band_cmap.FlushCache()

target_ds_cmap.GetRasterBand(1).WriteArray(tiles)
target_ds_cmap = None


(17730, 9203)
